In [10]:
import psycopg2
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

In [11]:
conn_string='host=summer22data.postgres.database.azure.com user=student_dennis dbname=daily_dennis password=dennis'

In [12]:
conn = psycopg2.connect(conn_string)
if conn.closed == 0:
    print("Connection established")

Connection established


In [13]:
fighters_query = 'SELECT * FROM fighters'
df_fighters = pd.read_sql(fighters_query, conn)
odds_query = 'SELECT * FROM odds'
df_odds = pd.read_sql(odds_query, conn)
fights_query = 'SELECT * from mma_fights'
df_fights = pd.read_sql(fights_query, conn)
conn.close()

C:\Users\AleksanderHalfstadAa\AppData\Local\Temp\ipykernel_17976\186172522.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fighters = pd.read_sql(fighters_query, conn)
C:\Users\AleksanderHalfstadAa\AppData\Local\Temp\ipykernel_17976\186172522.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_odds = pd.read_sql(odds_query, conn)
C:\Users\AleksanderHalfstadAa\AppData\Local\Temp\ipykernel_17976\186172522.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fights = pd.read_sql(fights_query, conn)


In [14]:
def scale_and_log_columns(dataframe, columns_to_scale, columns_to_log):
    
    transformed_df = dataframe.copy()
    scaler = StandardScaler()
    
    transformed_df[columns_to_scale] = scaler.fit_transform(transformed_df[columns_to_scale])

    transformed_df[columns_to_log] = transformed_df[columns_to_log].apply(lambda x: np.log1p(x) if x.min() > 0 else x)

    return transformed_df

Fighter Scaling

In [15]:
features_to_scale = [
    'r_avg_sig_str_landed',
    'r_avg_sig_str_landed',
    'r_avg_sig_str_pct',
    'r_avg_sub_att',
    'r_avg_td_landed',
    'r_avg_td_pct',
    'r_height_cms',
    'r_reach_cms', 
    'r_age']
features_to_log= ['r_avg_sig_str_landed', 'r_avg_sub_att', 'r_avg_td_landed']
processed_fighter =scale_and_log_columns(df_fighters, features_to_scale, features_to_log)

In [16]:
columns_to_fill_0 = [
    'r_avg_sig_str_landed',
    'r_avg_sig_str_pct',
    'r_avg_sub_att',
    'r_avg_td_landed',
    'r_avg_td_pct',
    'b_avg_sig_str_landed',
    'b_avg_sig_str_pct',
    'b_avg_sub_att',
    'b_avg_td_landed',
    'b_avg_td_pct'
]

df_fights[columns_to_fill_0] = df_fights[columns_to_fill_0].fillna(0)

Fight Scaling

In [17]:
features_to_scale = [
    'r_avg_sig_str_landed',
    'r_avg_sig_str_landed',
    'r_avg_sig_str_pct',
    'r_avg_sub_att',
    'r_avg_td_landed',
    'r_avg_td_pct',
    'r_height_cms',
    'r_reach_cms', 
    'r_age',
    'b_avg_sig_str_landed',
    'b_avg_sig_str_landed',
    'b_avg_sig_str_pct',
    'b_avg_sub_att',
    'b_avg_td_landed',
    'b_avg_td_pct',
    'b_height_cms',
    'b_reach_cms', 
    'b_age']
features_to_log= ['r_avg_sig_str_landed', 'r_avg_sub_att', 'r_avg_td_landed',
                  'b_avg_sig_str_landed', 'b_avg_sub_att', 'b_avg_td_landed']
processed_fights =scale_and_log_columns(df_fights, features_to_scale, features_to_log)

In [18]:
red_columns = []
blue_columns = []
for col in processed_fights.columns:
    if col.startswith('r_'):
        red_columns.append(col)
    elif col.startswith('b_'):
        blue_columns.append(col)

df_ufc_swapped = processed_fights.copy()

for r_col, b_col in zip(red_columns, blue_columns):
    df_ufc_swapped[[r_col, b_col]] = df_ufc_swapped[[b_col, r_col]]

winner_swap = {'Red': 'Blue', 'Blue': 'Red'}
df_ufc_swapped['winner'] = df_ufc_swapped['winner'].map(winner_swap)

df_ufc_combined = pd.concat([processed_fights, df_ufc_swapped], ignore_index=True)

In [19]:
df_ufc_combined.to_csv('../CSV Files/df_ufc_masters_w_reversed.csv', index=False)
processed_fighter.to_csv('../CSV Files/unique_fighters.csv', index=False)

ODDS TABELL TIL DATAFRAME

In [20]:
df_odds.columns

Index(['date', 'fighter_a', 'fighter_b', 'bookmaker', 'odds_a', 'odds_b'], dtype='object')

In [21]:

df_duplicated = df_odds.copy()

df_duplicated[['fighter_a', 'fighter_b']] = df_duplicated[['fighter_b', 'fighter_a']]
df_duplicated[['odds_a', 'odds_b']] = df_duplicated[['odds_b', 'odds_a']]

combined_df = pd.concat([df_odds, df_duplicated], ignore_index=True)

In [22]:
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Format the datetime objects to get only the date part
combined_df['date'] = combined_df['date'].dt.date

In [23]:
combined_df['date']

0        2020-12-31
1        2020-12-31
2        2020-12-31
3        2020-12-31
4        2020-12-31
            ...    
26487    2024-03-10
26488    2024-03-10
26489    2024-03-10
26490    2024-03-10
26491    2024-03-10
Name: date, Length: 26492, dtype: object

In [28]:
combined_df

,date,fighter_a,fighter_b,bookmaker,odds_a,odds_b,ev_a,ev_b
0,2020-12-31,Conor McGregor,Khabib Nurmagomedov,Unibet,250,-345,250.000000,28.985507
1,2020-12-31,Conor McGregor,Nate Diaz,Unibet,-278,205,35.971223,205.000000
2,2020-12-31,Conor McGregor,Jorge Masvidal,Unibet,135,-175,135.000000,57.142857
3,2020-12-31,Conor McGregor,Justin Gaethje,Unibet,-130,100,76.923077,100.000000
4,2020-12-31,Conor McGregor,Tony Ferguson,Unibet,-238,180,42.016807,180.000000
...,...,...,...,...,...,...,...,...
26487,2024-03-10,Sean O'Malley,Marlon Vera,LowVig.ag,-185,160,54.054054,160.000000
26488,2024-03-10,Sean O'Malley,Marlon Vera,BetOnline.ag,-185,160,54.054054,160.000000
26489,2024-03-10,Sean O'Malley,Marlon Vera,FanDuel,-215,164,46.511628,164.000000
26490,2024-03-10,Sean O'Malley,Marlon Vera,Caesars,-205,170,48.780488,170.000000


In [26]:
def calculate_ev(odds, bet_amount):
    if odds > 0:
        return (bet_amount / 100) * odds
    elif odds < 0:
        return (100 / abs(odds)) * bet_amount
    else:
        return 0

In [27]:
bet_amount=100
combined_df['ev_a'] = combined_df['odds_a'].apply(lambda x: calculate_ev(x, bet_amount))
combined_df['ev_b'] = combined_df['odds_b'].apply(lambda x: calculate_ev(x, bet_amount))

In [25]:
combined_df.to_csv('odds_reversed.csv', index=False)